# Is Dogecoin a joke? or can it be a new investment opportunity? 

The purpose of this project is to gain market insights for Dogecoin from 2015 to 2020 (current date) and to assess the strengths and weaknesses of the asset. In order to analyze this cryptocurrency the daily historical data for Dogecoin is downloaded along with Bitcoin and Litecoin for comparision from [CryptoCompare API](https://min-api.cryptocompare.com/documentation?key=Historical&cat=dataHistoday). I will demonstrate in USD currency and CCCAGG (Crypto Coin Comparison Aggregated Index) exchange rate as fixed factors for all three cryptocurrencies. Daily data from 2015-01-30 to 2020-07-22 is used since Cryptocompare API limits response to 2000 samples.

### Downloading the data

In [22]:
import requests
from datetime import datetime
import pandas as pd
from sklearn import preprocessing

import plotly as py
from plotly.offline import init_notebook_mode, iplot, plot
import plotly.graph_objs as go
from plotly.subplots import make_subplots

init_notebook_mode(connected=True)

In [3]:
# Constant factors
currency = 'USD'
exchange = 'CCCAGG'
time_interval = 'day'

# API for assets(Bitcoin, Litecoin, Dogecoin)
url_btc = 'https://min-api.cryptocompare.com/data/v2/histoday?fsym=BTC&tsym=USD&limit=2000'
url_ltc = 'https://min-api.cryptocompare.com/data/v2/histoday?fsym=LTC&tsym=USD&limit=2000'
url_doge = 'https://min-api.cryptocompare.com/data/v2/histoday?fsym=DOGE&tsym=USD&limit=2000'

The following columns will be extracted from the CryptoCompare API:

- <b>time</b> 
- <b>low</b>: the lowest price reached during the day
- <b>high</b>: the highest price reached during the day
- <b>open</b>: the price at the start of the day 
- <b>close</b>: the price at the end of the day
- <b>volumefrom</b>: the total amount of the base currency traded into the quote currency during the day
- <b>volumeto</b>: the total amount of the base currency traded into the quote currency during the day

In [7]:
# Format and create filename
def get_filename(asset, currency, exchange, datetime_interval, download_date):
    return '%s_%s_%s_%s_%s.csv' % (asset, currency, exchange, datetime_interval, download_date)

# Download API data into json format
def download_data(url):
    request = requests.get(url)
    data = request.json()
    return data

# Convert json formatted data to dataframe 
def convert_to_dataframe(data):
    df = pd.json_normalize(data=data['Data'], record_path='Data')
    df['time'] = pd.to_datetime(df.time, unit='s')
    df = df[['time', 'low', 'high', 'open',
             'close', 'volumefrom', 'volumeto']] # select only these columns
    return df

# Check if there is empty(None) data
def check_empty_data(df):
    indices = df[df.sum(axis=1) == 0].index
    print('There are %d empty datapoints' % indices.shape[0])
    df = df.drop(indices)
    return df

In [8]:
# Download data for 3 cryptocurrencies 
data_btc = download_data(url_btc)
data_ltc = download_data(url_ltc)
data_doge = download_data(url_doge)

In [9]:
# Convert json formatted file to dataframe
df_btc = convert_to_dataframe(data_btc)
df_ltc = convert_to_dataframe(data_ltc)
df_doge = convert_to_dataframe(data_doge)

In [10]:
#Check if there's empty data in each dataframe
df_btc = check_empty_data(df_btc)
df_ltc = check_empty_data(df_ltc)
df_doge = check_empty_data(df_doge)
# All 3 cryptocurrencies have no empty datapoints 

There are 0 empty datapoints
There are 0 empty datapoints
There are 0 empty datapoints


In [11]:
#Create current date
current_datetime = datetime.now().date().isoformat()

#Create filenames
file_btc = get_filename('Bitcoin', currency, exchange, time_interval, current_datetime)
file_ltc = get_filename('Litecoin', currency, exchange, time_interval, current_datetime)
file_doge = get_filename('Dogecoin', currency, exchange, time_interval, current_datetime)

print('Save data as %s' % file_btc)
print('Save data as %s' % file_ltc)
print('Save data as %s' % file_doge)

#Save datasets in csv formatted files
df_btc.to_csv(file_btc, index=False)
df_ltc.to_csv(file_ltc, index=False)
df_doge.to_csv(file_doge, index=False)

Save data as Bitcoin_USD_CCCAGG_day_2020-07-23.csv
Save data as Litecoin_USD_CCCAGG_day_2020-07-23.csv
Save data as Dogecoin_USD_CCCAGG_day_2020-07-23.csv


### Loading the data

In [13]:
def read_dataset(filename):
    print('Read data from %s' % filename)
    df = pd.read_csv(filename)
    df.time = pd.to_datetime(df.time)
    df = df.set_index('time') 
    df = df.sort_index() # sort by time
    print(df.shape)
    return df

# Use saved csv files 
df_btc = read_dataset(file_btc)
df_ltc = read_dataset(file_ltc)
df_doge = read_dataset(file_doge)

Read data from Bitcoin_USD_CCCAGG_day_2020-07-23.csv
(2001, 6)
Read data from Litecoin_USD_CCCAGG_day_2020-07-23.csv
(2001, 6)
Read data from Dogecoin_USD_CCCAGG_day_2020-07-23.csv
(2001, 6)


In [14]:
df_btc.head()

,low,high,open,close,volumefrom,volumeto
time,,,,,,
2015-01-31,216.46,235.36,231.84,218.51,42154.24,9645307.86
2015-02-01,209.71,231.44,218.51,225.29,41163.75,9058010.18
2015-02-02,220.53,242.51,225.29,238.86,39015.62,8965813.60
2015-02-03,223.91,247.48,238.86,227.41,63078.64,15000263.95
2015-02-04,220.23,230.71,227.41,226.58,55242.53,12495299.13


In [15]:
df_ltc.head()

,low,high,open,close,volumefrom,volumeto
time,,,,,,
2015-01-31,1.850,1.940,1.92,1.88,228360.61,435849.87
2015-02-01,1.660,1.880,1.88,1.84,567802.67,1002001.92
2015-02-02,1.730,1.850,1.84,1.78,490350.58,874158.35
2015-02-03,1.781,1.965,1.78,1.83,643232.70,1215922.22
2015-02-04,1.750,1.850,1.83,1.80,459769.70,824996.67


In [16]:
df_doge.head()

,low,high,open,close,volumefrom,volumeto
time,,,,,,
2015-01-31,0.000140,0.000151,0.000140,0.000140,270.59,0.03790
2015-02-01,0.000134,0.000150,0.000140,0.000144,553.07,0.07493
2015-02-02,0.000141,0.000151,0.000144,0.000141,1982.11,0.27290
2015-02-03,0.000141,0.000158,0.000141,0.000145,34906.03,5.23300
2015-02-04,0.000140,0.000151,0.000145,0.000145,26276.36,3.92700


### Extracting closing price data for all 3 cryptocurrencies

In [17]:
# Create dataframe that shows the closing prices of all 3 cryptocurrencies
df_close = pd.DataFrame({'BTC': df_btc.close,
                         'LTC': df_ltc.close,
                         'DOGE': df_doge.close})

In [18]:
df_close.head()

,BTC,LTC,DOGE
time,,,
2015-01-31,218.51,1.88,0.000140
2015-02-01,225.29,1.84,0.000144
2015-02-02,238.86,1.78,0.000141
2015-02-03,227.41,1.83,0.000145
2015-02-04,226.58,1.80,0.000145


In [19]:
df_close.describe()

,BTC,LTC,DOGE
count,2001.000000,2001.000000,2001.000000
mean,4474.731439,47.208289,0.001872
std,4019.058565,54.511759,0.001913
min,211.430000,1.320000,0.000015
25%,574.230000,3.812000,0.000221
50%,3842.940000,41.060000,0.001927
75%,7899.110000,62.360000,0.002699
max,19345.490000,357.510000,0.017000


In [20]:
fig = make_subplots(rows=3, cols=1,
                    subplot_titles=("Dogecoin", "Litecoin", "Bitcoin"),
                    shared_xaxes=False, 
                    vertical_spacing=0.1)

fig.add_trace(go.Scatter(x=df_close.index, y=df_close.BTC,name='Bitcoin'),
              row=3, col=1)

fig.add_trace(go.Scatter(x=df_close.index, y=df_close.LTC,name='Litecoin'),
              row=2, col=1)

fig.add_trace(go.Scatter(x=df_close.index, y=df_close.DOGE,name='Dogecoin'),
              row=1, col=1)

fig.update_layout(height=1000, width=800,
                  title="Fig1. Daily closing price of Dogecoin, Litecoin and Bitcoin")

fig.update_xaxes(title_text="Time", tickformat='%d %b, %Y', row=3, col=1)
fig.update_xaxes(tickformat='%d %b, %Y', row=2, col=1)
fig.update_xaxes(tickformat='%d %b, %Y', row=1, col=1)
fig.update_yaxes(title_text="Price($)", row=2, col=1)
fig.update_yaxes(tickprefix="$")

fig.show()

From the graphs above, we could find that there is a significant relationship between the prices of Litecoin and Dogecoin (which both are altcoins), and the price of Bitcoin. The price of Bitcoin reached the highest peak of nearly **\\$20,000** in December 2017. After the price spike of cryptocurrencies in 2017. The price of Bitcoin then crashed all and reached a low of **\\$6,800** by April 2018 and a low of **\\$3,500** by December 2018. This means the price of Bitcoin fell by about **83 percent** within a year. 

Bitcoin prices showed an increase of a multiple of four in 2018, reaching over **\\$10,000** by June 2019. However, the price of bitcoin fell down to around **\\$6,600** by the end of the year. In 2020, the highest point of bitcoin has been under **\\$10,000** when the market crashed in March.

We can also see this price trend in Dogecoin. When the 2018 cryptocurrency crash happened, Dogecoin reached a peak of **\\$0.017** per Dogecoin in January 2018. The price of Dogecoin crashing fell back down to a low of **\\$0.0028** by April 2018 and below **\\$0.0021** by December 2018. As the price of Dogecoin fell by about **88 percent**, Dogecoin lost its value more than Bitcoin during the year. 

The price of Dogecoin resurged to around \\$0.006 by April 2019 and reached to the hightest peak at \\$0.007 by September 2019 after the 2018 cryptocurrency crash. However, like Bitcoin, Dogecoin price fell to around $0.002 in December. 

On top of that, since Dogecoin has adopted merged-mining with Litecoin in 2014, we can find a similarity of the price trend between Litecoin and Dogecoin after April 2019. Litecoin reached to its highest value of **\\$82.98** in February 2020 and subsequently fell down to its lowest point of **\\$30.49** in March. It has since stablized around **\\$45.** Similarly, Dogecoin price surged to **\\$0.0034** in February 2020 from the low point of **\\$0.00212** at the start of the year and has then dropped to **\\$0.0015** in March. It has been steadily recovering since.

You could find the strong correlated relationships between these cryptocurrencies on Fig2.(below) by putting the different cryptocurrency datasets on the same scale(between 0 and 100). 

### Checking the correlation between all 3 cryptocurrencies

In [23]:
# Putting all 3 cryptocurrencies on the same scale(0, 100)
scaler = preprocessing.MinMaxScaler(feature_range=(0, 100))
scaled_close_price = scaler.fit_transform(df_close)
scaled_close_price

array([[3.70020790e-02, 1.57219462e-01, 7.35237840e-01],
       [7.24362733e-02, 1.45989500e-01, 7.59965570e-01],
       [1.43356925e-01, 1.29144558e-01, 7.39947883e-01],
       ...,
       [4.79845887e+01, 1.19234117e+01, 1.93404997e+01],
       [4.87393162e+01, 1.22771555e+01, 1.92933992e+01],
       [4.86022308e+01, 1.22715405e+01, 1.92757366e+01]])

In [24]:
# Convert the scaled data to a dataframe
df_close_scaled = pd.DataFrame(scaled_close_price, columns = df_close.columns)

In [25]:
df_close_scaled.head()

,BTC,LTC,DOGE
0,0.037002,0.157219,0.735238
1,0.072436,0.145989,0.759966
2,0.143357,0.129145,0.739948
3,0.083516,0.143182,0.765264
4,0.079178,0.134760,0.765264


In [26]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_close_scaled.index, y=df_close_scaled.BTC,
                    mode='lines',
                    name='Bitcoin'))
fig.add_trace(go.Scatter(x=df_close_scaled.index, y=df_close_scaled.LTC,
                    mode='lines',
                    name='Litecoin'))
fig.add_trace(go.Scatter(x=df_close_scaled.index, y=df_close_scaled.DOGE,
                    mode='lines', name='Dogecoin'))

fig.update_layout(title_text="Fig2. The similarity of the price trend between Bitcoin, Litecoin and Dogecoin",\
                  hovermode='x unified')
fig.update_xaxes(title_text='Days')
fig.update_yaxes(title_text='Scaled Price')
fig.update_yaxes(tickprefix="$")

fig.show()


As we figured out in Fig 1., we can clearly see the strong correlated relationship for the price movements in all 3 cryptocurrencies. 

### Calculating Daily Simple Return

In [27]:
# Calculating the percentage change of the daily price  
daily_simple_return= df_close.pct_change()
daily_simple_return.head()

,BTC,LTC,DOGE
time,,,
2015-01-31,NaN,NaN,NaN
2015-02-01,0.031028,-0.021277,0.030021
2015-02-02,0.060233,-0.032609,-0.023595
2015-02-03,-0.047936,0.028090,0.030561
2015-02-04,-0.003650,-0.016393,0.000000


In [28]:
daily_simple_return.describe()

,BTC,LTC,DOGE
count,2000.000000,2000.000000,2000.000000
mean,0.002654,0.003271,0.016941
std,0.038891,0.059608,0.285078
min,-0.381176,-0.439695,-0.911000
25%,-0.010960,-0.019436,-0.024798
50%,0.001974,-0.000327,0.000000
75%,0.017452,0.019689,0.024948
max,0.255605,0.737439,9.537408


In [29]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=daily_simple_return.index, y=daily_simple_return.BTC,
                    mode='lines',
                    name='Bitcoin'))
fig.add_trace(go.Scatter(x=daily_simple_return.index, y=daily_simple_return.LTC,
                    mode='lines',
                    name='Litecoin'))
fig.add_trace(go.Scatter(x=daily_simple_return.index, y=daily_simple_return.DOGE,
                    mode='lines', name='Dogecoin'))

fig.update_layout(title_text="Fig3. Daily simple return of Bitcoin, Litecoin and Dogecoin",\
                  hovermode='x unified')
fig.update_xaxes(title_text='Time', tickformat='%d %b,%Y')
fig.update_yaxes(title_text='Percentage')
fig.update_layout(yaxis=dict(range=[-1,10]))
fig.show()

Dogecoin shows the daily simple returns of Dogecoin fluctuated the most compared to Bitcoin and Litecoin. There are several moments where Dogecoin's daily return was exceeded both Bitecoin and Litecoin. From this result, we can assume that Dogecoin is the most volatile cryptocurrency among three cryptocurrencies, but also it gives the highest daily return. We can prove this by calculating the volatility and the average of the cryptocurrency below. 

In [32]:
# Calculate the volatility of 3 cryptocurrencies
print('The volatility of cryptocurrency :')
daily_simple_return.std()

The volatility of cryptocurrency :


BTC     0.038891
LTC     0.059608
DOGE    0.285078
dtype: float64

In [33]:
# Show the mean or average daily simple return
print('The average return of cryptocurrency :')
daily_simple_return.mean()

The average return of cryptocurrency :


BTC     0.002654
LTC     0.003271
DOGE    0.016941
dtype: float64

We find that the volatility of Dogecoin is nearly five times more than Litecoin and seven times more than Bitcoin, while the average return of Dogecoin is about five time higher than Litecoin and six times higher than Bitcoin. It means Dogecoin can be very volatile but there are possiblities to earn the higher daily profit than other top cryptocurrencies. 

### Calculating Daily Cumulative simple returns 

In [34]:
# Create daily cumulative simple returns for 3 cryptocurrencies 
cumulative_return = (daily_simple_return+1).cumprod()
cumulative_return.head()

,BTC,LTC,DOGE
time,,,
2015-01-31,NaN,NaN,NaN
2015-02-01,1.031028,0.978723,1.030021
2015-02-02,1.093131,0.946809,1.005718
2015-02-03,1.040730,0.973404,1.036455
2015-02-04,1.036932,0.957447,1.036455


In [35]:
cumulative_return.describe()

,BTC,LTC,DOGE
count,2000.000000,2000.000000,2000.000000
mean,20.488122,25.122847,13.388572
std,18.392457,28.997852,13.672255
min,0.967599,0.702128,0.107362
25%,2.629067,2.027660,1.583810
50%,17.592902,21.843085,13.791994
75%,36.170713,33.172872,19.294139
max,88.533660,190.164894,121.515368


In [36]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=cumulative_return.index, y=cumulative_return.BTC,
                    mode='lines',
                    name='Bitcoin'))
fig.add_trace(go.Scatter(x=cumulative_return.index, y=cumulative_return.LTC,
                    mode='lines',
                    name='Litecoin'))
fig.add_trace(go.Scatter(x=cumulative_return.index, y=cumulative_return.DOGE,
                    mode='lines', name='Dogecoin'))

fig.update_layout(title_text="Fig4. Cumulative Daily Simple return of Bitcoin, Litecoin and Dogecoin",\
                  hovermode='x unified')
fig.update_xaxes(title_text='Time', tickformat='%d %b,%Y')
fig.update_yaxes(title_text='Growth of $1 investment')

fig.show()

If we had invested **one dollar** in all three assets and sold on 19 December 2017, our investment would have grown to around \\$185 for Litecoin, around \\$81 for Bitcoin, and over \\$38 for Dogecoin. Futhermore, if we sold today (23 July 2020), we would earn nearly \\$44 for Bitcoin, almost \\$24 for Litecoin and over \\$24 Dogecoin. It shows that Dogecoin has the capability for the investment since there were moments where Dogecoin's cumulative daily return had exceeded the cumulative daily return of other currencies in 2017 and 2018. 

### Interesting events in July 

In [37]:
# Select Dogecoin data only in July
df_close_july = df_close[(df_close.index > '2020-06-30') & (df_close.index <= '2020-07-22')]
df_close_doge_july = df_close_july['DOGE']

In [38]:
fig = go.Figure(data=go.Scatter(x=df_close_doge_july.index, y=df_close_doge_july.values,mode='markers+lines'))
fig.update_yaxes(tickprefix="$")
fig.update_layout(title_text="Fig5. The price change of Dogecoin in July")
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Price')
fig.show()

There were two interesting events that affected the resurgence of Dogecoin prices in July. 
The first event was a sudden surge TikTok (social media platform) viral videos regarding Dogecoin. In the video, one of the Tiktok users asked followers to put **\\$25** into Dogecoin in order to push its price to **$1** in July 6. Because of this spontaneous viral event, the price of Dogecoin has nearly doubled since July 6, rising **80.6%** to **\\$0.00475** from **\\$0.00263**.

Another event was a tweet about Dogecoin by Elon Musk (Tesla Chief Executive). Musk showed an affinity with Dogecoin since last year. Dogecoin’s price jump started on July 17 when Mask said that it is inevitable that Dogecoin standard would take over the global financial system. After this tweet, its price rose by nearly **14%**, from **\\$0.003056** to **\\$0.003486**. 

These two joke-like events pushed the price of Dogecoin in July, and it shows that Dogecoin can be very volatile from social media, especially memes. 

## Conclusion

Dogecoin, a meme-based cryptocurrency, has a market capitalization of **\\$411 million** at current date (23 July 2020) and is currently a top 30 cryptocurrency, according to cryptocurrency tracker [CoinMarketCap](https://coinmarketcap.com). From analysis, I could say Dogecoin has investment potential even though it has started as a joke. We saw its daily return fluctuates the most in Fig3, but it was designed to be inflationary. Unlike Bitcoin, Dogecoin can be mined at 10,000 coins per minute, which remains cheaper prices. This makes Dogecoin faster and cheaper to use than other top cryptocurrencies, while it makes less appealing to investors due to a extremely high supply of currency. 

However, there is no technical updates yet and there might be a possibility that Dogecoin can become the part of the [Ethereum Ecosystem](https://blokt.com/news/dogecoin-doge-interview-core-dev-ross-nicoll-says-huge-step-forward-made-on-doge-ethereum-bridge-exclusive). When technical improvement happens, we could expect the increase of the price. Furthermore, because of its user-friendly traits, Dogecoin is commonly used for tipping service; users can pay tips in Dogecoin to other people who post enjoyable and fun contents. You can check the Dogecoin reddit community from [here](https://www.reddit.com/r/dogecoin/). 

In conlusion, even though there are weak factors of Dogecoin such as high volatilaty, low price, and lack of technology, its strengths like convenient usage, easy to approach, and high return would make investment in Dogecoin interesting. There has been already two price pumps in July and maybe the price can "go to the moon" in the future. 

According to the strengths and weaknesses of Dogecoin, I would rate it **61 points** using the *weighted average method*.